<a href="https://colab.research.google.com/github/Nilanshrajput/AdvitiyaHackathon/blob/master/BertV2_fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 481kB 3.0MB/s 
     |████████████████████████████████| 3.1MB 20.2MB/s 
     |████████████████████████████████| 1.0MB 20.7MB/s 
     |████████████████████████████████| 870kB 19.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=bd92cf468d52fcd4b1b2535bb620b032b6ce6acd5d0ae59825b97044e1c980e2
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import torch
import numpy as np
import pandas as pd
from transformers import BertModel, BertTokenizer, BertForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from torch import nn

from time import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
drive_pt="/content/drive/My Drive/dl_projects/AdvitiyaHackathon/"

git_path = "AdvitiyaHackathon/"
path = drive_pt+"ScrappeData"

In [0]:
df1 = pd.read_csv(drive_pt+"claims_final_train_clean.csv",index_col=0)
df2 = pd.read_csv(drive_pt+"claims_final_val_clean.csv", index_col= 0 )

In [41]:
df1.head()

,text,cat
0,agricultural autonomous vehicle platform arti...,3
1,drive circuit semiconductor type light source...,2
2,modular electronic carbonator fluid level con...,0
3,natural stable solution alpha acid use improv...,1
4,par bake mill coffee bean use food beverage d...,0


In [0]:
df = df1.append(df2, ignore_index =True).sample(frac=1).reset_index(drop=True)


In [57]:

df.shape

(1199, 2)

In [0]:
df.to_csv(drive_pt+"train/claims_train_all.csv")


In [49]:
df.head()

,text,cat
0,control device start engine vehicle perform o...,2
1,engineer microorgansims method improve crotyl...,1
2,invert pendulum type vehicle perform operatio...,2
3,vehicle body structure perform operation tran...,2
4,refrigerator equip apparatus produce carbonat...,0


In [0]:
df1 = df1.sample(frac=1).reset_index(drop=True)
df2 = df2.sample(frac=1).reset_index(drop=True)


In [0]:
"""df1.to_csv(drive_pt+"final_train_clean.csv")
df2.to_csv(drive_pt+"final_val_clean.csv")"""

'df1.to_csv(drive_pt+"final_train_clean.csv")\ndf2.to_csv(drive_pt+"final_val_clean.csv")'

In [0]:
#df2.head()

In [0]:
bert_model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [59]:

from keras.preprocessing.sequence import pad_sequences


class LoadDataset(Dataset):

    def __init__(self, filename, maxlen):

        # Store the contents of the file in a pandas dataframe
        self.df = pd.read_csv(filename, delimiter=',',index_col=0)

        # Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # Define the Maxlength for padding/truncating
        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        # Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        #sentence = re.sub(' +', ' ',sentence)
        label = torch.tensor((self.df.loc[index, 'cat']))

        # Tokenize the sentence
        tokens_ids = self.tokenizer.encode(str(sentence),add_special_tokens = True,max_length = self.maxlen, pad_to_max_length=True)
        """
        tokens_ids = pad_sequences([tokens_ids], maxlen=self.maxlen, dtype="long", 
                          value=0, truncating="post", padding="post")
        """   
        # Inserting the CLS and SEP token at the beginning and end of the sentence
        #tokens = ['[CLS]'] + tokens 
        """
        if len(tokens_ids) < self.maxlen:
            tokens_ids = [tokens_ids + [0] for _ in range(self.maxlen - len(tokens_ids))]
        else:
            tokens_ids = tokens_ids[:self.maxlen-1]         
        """  
        # Padding/truncating the sentences to the maximum length


        # Convert the sequence to ids with BERT Vocabulary
        #tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        
        # Converting the list to a pytorch tensor
        tokens_ids_tensor = torch.tensor(tokens_ids)

        # Obtaining the attention mask
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

Using TensorFlow backend.


In [0]:
# Creating instances of training and validation set
train_set = LoadDataset(filename = drive_pt+'train/claims_train_all.csv', maxlen = 512)
val_set = LoadDataset(filename = drive_pt+'classification_final_val_clean.csv', maxlen = 512)

In [0]:

# Creating intsances of training and validation dataloaders
train_loader = DataLoader(train_set, batch_size = 8, shuffle = True, num_workers = 5)
val_loader = DataLoader(val_set, batch_size = 8, shuffle = True, num_workers = 5)

In [0]:
#batch = next(iter(train_loader))

In [0]:
#batch[0].shape

In [0]:

class PatentClassifier(nn.Module):

    def __init__(self, freeze_bert = True):
        super(PatentClassifier, self).__init__()

        # Instantiating the BERT model object 
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        # Defining layers like dropout and linear
        self.dropout = nn.Dropout(0.15)
        self.classifier = nn.Linear(768, 4)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        # Getting contextualized representations from BERT Layer
        cont_reps, _ = self.bert_layer(seq, attention_mask = attn_masks)

        # Obtaining the representation of [CLS] head
        cls_rep = cont_reps[:, 0]
        # print('CLS shape: ',cls_rep.shape)

        # Feeding cls_rep to the classifier layer
        logits = self.classifier(cls_rep)
        # print('Logits shape: ',logits.shape)

        return logits
        
        

In [64]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 4, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = True, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

#model = PatentClassifier(freeze_bert= False)

# Tell pytorch to run this model on the GPU.
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [65]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [0]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [0]:

from transformers import get_linear_schedule_with_warmup

# Number of training epochs (authors recommend between 2 and 4)
epochs = 20

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_loader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:

import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [71]:


import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [77]:
# Saving our model
import os
save_path = drive_pt+ "Models/"


# Loading the checkpoints for resuming training
checkpoint = torch.load(save_path+'bertv4_claims_512_8_20.pth')
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [0]:
import random

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128


# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# Store the average loss after each epoch so we can plot them.
loss_values = []

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_loader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_loader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
       
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # This will return the loss (rather than the model output) because we
        # have provided the `labels`.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        
        # The call to `model` always returns a tuple, so we need to pull the 
        # loss value out of the tuple.
        loss = outputs[0]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_loader)            
    
    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in val_loader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # Telling the model not to compute or store gradients, saving memory and
        # speeding up validation
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have
            # not provided labels.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        logits = outputs[0]

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # Calculate the accuracy for this batch of test sentences.
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        # Accumulate the total accuracy.
        eval_accuracy += tmp_eval_accuracy

        # Track the number of batches
        nb_eval_steps += 1

    # Report the final accuracy for this validation run.
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 20 ========
Training...
  Batch    40  of    150.    Elapsed: 0:00:36.
  Batch    80  of    150.    Elapsed: 0:01:11.


In [0]:
# Saving our model
import os
save_path = drive_pt+ "Models/"
if not os.path.isdir(save_path):
    os.mkdir(save_path)

torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}, save_path+'finalbertv4_claims_512_8_20.pth')

ERROR: Could not find a version that satisfies the requirement bertviz (from versions: none)
ERROR: No matching distribution found for bertviz


In [0]:
 # Initialize the BERT tokenizer
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

        # Define the Maxlength for padding/truncating
        self.maxlen = maxlen

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):

        # Selecting the sentence and label at the specified index in the data frame
        sentence = self.df.loc[index, 'text']
        #sentence = re.sub(' +', ' ',sentence)
        label = torch.tensor((self.df.loc[index, 'cat']))

        # Tokenize the sentence
        tokens_ids = self.tokenizer.encode(str(sentence),add_special_tokens = True,max_length = self.maxlen, pad_to_max_length=True)
        """
        tokens_ids = pad_sequences([tokens_ids], maxlen=self.maxlen, dtype="long", 
                          value=0, truncating="post", padding="post")
        """   
        # Inserting the CLS and SEP token at the beginning and end of the sentence
        #tokens = ['[CLS]'] + tokens 
        """
        if len(tokens_ids) < self.maxlen:
            tokens_ids = [tokens_ids + [0] for _ in range(self.maxlen - len(tokens_ids))]
        else:
            tokens_ids = tokens_ids[:self.maxlen-1]         
        """  
        # Padding/truncating the sentences to the maximum length


        # Convert the sequence to ids with BERT Vocabulary
        #tokens_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        
        # Converting the list to a pytorch tensor
        tokens_ids_tensor = torch.tensor(tokens_ids)

        # Obtaining the attention mask
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

Evaluation

In [0]:
def pre_process(sentence, maxlen):
  tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

  tokens_ids = tokenizer.encode(str(sentence),add_special_tokens = True,max_length = maxlen, pad_to_max_length=True)
  # Converting the list to a pytorch tensor
  tokens_ids_tensor = torch.tensor(tokens_ids)

  # Obtaining the attention mask
  attn_mask = (tokens_ids_tensor != 0).long()

  return tokens_ids_tensor, attn_mask


In [0]:
df2 = pd.read_csv(drive_pt+"claims_final_val_clean.csv", index_col= 0 )

In [0]:
index = 20
sentence = df2.loc[index, 'text']
sentence = '''
The invention relates to a cross-flow filtration system (1) for filtering a suspension, in particular a wine, comprising at least three substantially cylindrical hollow fibre modules (3 – 22) for retaining a retentate of the suspension; a circulation pump (23), fluidically connected to the hollow fibre modules (3 – 22), for generating a mechanical flow of the suspension; a flow distributor (25) for distributing the suspension to the hollow fibre modules (3 – 22); and a flow collector (30) for collecting the suspension from the hollow fibre modules (3 – 22), said flow distributor (25) and flow collector (30) being fluidically connected to the circulation pump (23) and to the hollow fibre modules (3, 4, 5, 6), and being substantially centrally arranged on a longitudinal axis (24) of the cross-flow filtration system (1). A homogenous flow is produced when the hollow fibre modules (3 – 22) are arranged in an axially parallel manner about the longitudinal axis (24

1 . Cross-flow filtration system (1, 2) for filtering a suspension, in particular a wine, with
 at least three substantially cylindrical hollow fiber modules (3-22) for retaining a retentate of the suspension,
 a circulating pump (23) fluidically connected to the hollow fiber modules (3-22) for producing a mechanical flow of the suspension,
 - A flow divider (25, 26) for distributing the suspension to the hollow fiber modules (3 - 22) and
 a flow collector (30, 31) for collecting the suspension from the hollow fiber modules (3 - 22),
wherein the flow divider (25, 26) and the flow collector (30, 31) with the circulation pump (23) and the hollow fiber modules (3 - 22) fluidly connected and substantially centrally on a longitudinal axis (24) of the Querstromfiltrationsys- system (1, 2 ) are arranged
characterized,
the hollow fiber modules (3 - 22) are arranged axially parallel to the longitudinal axis (24).
2. Cross-flow filtration system according to claim 1,
characterized,
in that the flow divider (25, 26) comprises a first rotary body (27) with a first tip (28) and a first circumferential lateral surface (29), arranged such that the flow divider (25, 26) substantially separates the suspension evenly distributed to the hollow fiber modules (3 - 22) when the circulation pump (23) directs the flow to the first tip (28).
3. cross-flow filtration system according to claim 2,
characterized,
the first lateral surface (29) is concave.
4. cross-flow filtration system according to claim 2,
characterized,
the first rotary body (27) is conical, in particular a straight circular cone.
5. cross-flow filtration system according to claim 3 or 4,
characterized,
the first rotary body (27) has depressions of the first lateral surface (29) arranged uniformly around the first tip (28) for guiding the suspension along the first lateral surface (29).
6. cross-flow filtration system according to one of claims 1 to 5,
characterized,
in that the flow collector (30, 31) comprises a second rotary body (32) with a second tip (33) and a second circumferential lateral surface (34) arranged such that the flow collector (30) feeds the suspension to the circulation pump (23) the hollow fiber modules (3 - 22) direct the flow to the second lateral surface (34).
7. cross-flow filtration system according to claim 6,
characterized,
the second lateral surface (34) is concave.
8. cross-flow filtration system (1, 2) according to claim 6,
characterized,
the second rotary body (32) is conical, in particular a straight circular cone.
9. cross-flow filtration system according to claim 7 or 8,
characterized,
the second rotary body (32) has depressions of the second lateral surface (34) arranged uniformly around the second tip (33) for guiding the suspension along the second lateral surface (34).
10. cross-flow filtration system according to one of claims 1 to 9,
marked by
a ventilator (35, 36) fluidly connected to the hollow fiber modules (3 - 22) and the circulation pump (23) for venting the crossflow filtration system (1, 2).
1 1. Cross-flow filtration system according to one of claims 1 to 10,
marked by
at least one mechanically connected to the hollow fiber modules (3 - 22) and the circulation pump (23) rack (37) for holding the hollow fiber modules (3 - 22).
12. Cross-flow filtration system according to claim 1 1,
marked by
a plurality of racks (37), wherein each rack (37) may consist of one or more hollow fiber modules (7 - 22).
13. Cross-flow filtration system according to one of claims 1 to 12, characterized
the transverse fiber modules (3 - 22) are arranged equidistant from the longitudinal axis (24).
14. Cross-flow filtration system according to one of claims 1 to 13,
characterized,
the transverse fiber modules (3 - 22) are arranged rotationally symmetrically about the longitudinal axis (24).
15. Cross-flow filtration system according to one of claims 1 to 14,
 '''



label = df2.loc[index, 'cat']
print(label)

2


In [0]:
b_input_ids, b_input_mask = pre_process(sentence, maxlen = 512)

In [0]:
b_input_mask = b_input_mask.unsqueeze(dim = 0).to(device)
b_input_ids = b_input_ids.unsqueeze(dim = 0).to(device)
b_input_mask.shape

torch.Size([1, 512])

In [0]:
model.eval()

In [0]:
outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)

In [0]:
logits = outputs[0]

# Move logits and labels to CPU
logits = logits.detach().cpu().numpy()
np.argmax(logits, axis=1).flatten()

array([1])

In [0]:
while(1):
  a=1


KeyboardInterrupt: ignored

In [0]:
  str()

NameError: ignored

In [0]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
import re
location = "d125sd{g5d"
letters_only = re.sub("[^a-zA-Z]",  # Search for all non-letters
                          " ",          # Replace all non-letters with spaces
                          str(location))

In [0]:
letters_only

'd   sd g d'

In [0]:
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

print(device)

cuda


In [0]:
# Defining a function for calculating accuracy
def logits_accuracy(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    preds = (probs > 0.5).long()
    acc = (preds.squeeze() == labels).float().mean()
    return acc

In [0]:
# Defining an evaluation function for training 
def evaluate(net, criterion, val_loader, device):
  
    losses, accuracies = 0, 0
    
    # Setting model to evaluation mode
    net.eval()

    count = 0
    for (seq, attn_masks, labels) in val_loader:
        count += 1

        # Move inputs and targets to device
        seq, attn_masks, labels = seq.to(device), attn_masks.to(device), labels.to(device)

        # Get logit predictions
        val_logits = net(seq, attn_masks)

        # Calculate loss
        val_loss = criterion(val_logits.squeeze(-1), labels.float())
        losses += val_loss.item()

        # Calculate validation accuracy
        accuracies += logits_accuracy(val_logits, labels)

    return losses / count, accuracies / count

In [0]:
def train(net, criterion, optimizer, train_loader, val_loader, device, epochs=4, print_every=100):
    
    # Move model to device
    net.to(device)
    # Setting model to training mode
    net.train()

    print('========== ========== STARTING TRAINING ========== ==========')

    for epoch in range(epochs):

        print('\n\n========== EPOCH {} =========='.format(epoch))        
        t1 = time()

        for i, (seq, attn_masks, labels) in enumerate(train_loader):

            # Clear gradients
            optimizer.zero_grad()  

            # Moving tensors to device
            seq, attn_masks, labels = seq.to(device), attn_masks.to(device), labels.to(device)

            # Obtaining the logits from the model
            logits = net(seq,attn_masks)

            # Calculating the loss
            loss = criterion(logits.squeeze(-1), labels.float())

            # Backpropagating the gradients
            loss.backward()

            # Clipping gradients to tackle exploding gradients
            nn.utils.clip_grad_norm_(net.parameters(), 1)

            # Optimization step
            optimizer.step()

            if (i + 1) % print_every == 0:
                print("Iteration {} ==== Loss: {}".format(i+1, loss.item()))

        t2 = time()
        print('Time Taken for Epoch: {}'.format(t2-t1))
        print('\n========== Validating ==========')
        mean_val_loss, mean_val_acc = evaluate(net, criterion, val_loader, device)
        print("Validation Loss: {}\nValidation Accuracy: {}".format(mean_val_loss, mean_val_acc))
